In [175]:
import pandas as pd
data = pd.read_csv("CarPrice_Assignment.csv")
data.drop('car_ID', axis=1, inplace=True)

In [191]:
# data.columns
# data.isnull().sum()
# data.describe()
# data.info()

In [177]:
import seaborn as sns
import matplotlib.pyplot as plt

In [192]:
# variable = 'wheelbase'
# sns.scatterplot(x=variable, y='price', data=data)

In [193]:
data.select_dtypes(include='number').corr()['price'].sort_values(ascending=False)
data.select_dtypes(include='number').corr(method='spearman')['price'].sort_values(ascending=False)
print()

In [180]:
# Got from DeepSeek

from sklearn.model_selection import KFold

def kfold_target_encoding(df, categorical_col, target_col, n_folds=5, smooth=0):
    new_col = categorical_col + "_encoded"
    df[new_col] = np.nan

    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for train_idx, val_idx in kf.split(df):
        X_train, X_val = df.iloc[train_idx], df.iloc[val_idx]
        
        # Calculate mean target per category (with smoothing)
        means = X_train.groupby(categorical_col)[target_col].mean()
        counts = X_train.groupby(categorical_col)[target_col].count()
        
        # Smoothing to handle rare categories
        global_mean = X_train[target_col].mean()
        smooth_means = (means * counts + global_mean * smooth) / (counts + smooth)
        
        # Apply encoding to validation fold
        df.loc[val_idx, new_col] = X_val[categorical_col].map(smooth_means)
    
    # Fill remaining NaNs with global mean (if any)
    df[new_col].fillna(global_mean, inplace=True)
    
    return df

for column in data.select_dtypes(include='object').columns:
    data = kfold_target_encoding(data, column, 'price', n_folds=3, smooth=1)

In [194]:
correlation = data.select_dtypes(include='number').corr()['price'].sort_values(ascending=False)
selected_features = correlation[abs(correlation) > 0.6].keys().drop('price')
# selected_features
# data.select_dtypes(include='number').corr(method='spearman')['price'].sort_values(ascending=False)

In [195]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# selected_features = ['stroke', 'peakrpm', 'symboling']
# selected_features = ['horsepower', 'enginesize', 'citympg', 'highwaympg', 'wheelbase']
# selected_features = ['price']
# selected_features += ['cylindernumber_encoded', 'doornumber_encoded']

X = data[selected_features]
y = data['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LinearRegression()
model.fit(X_train, y_train)

# Check model performance
print(f"R² Score: {model.score(X_test, y_test)}")

R² Score: 0.8563382543264395


In [196]:
to_predict = data[selected_features].iloc[[0]]
print(to_predict)
model.predict(to_predict)

   enginesize  curbweight  horsepower  carwidth  cylindernumber_encoded  \
0         130        2548         111      64.1            10107.704501   

   carlength  drivewheel_encoded  citympg  highwaympg  
0      168.8        20868.566349       21          27  


array([13594.69050895])